# Dark current: the ideal case

In [ ]:
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

from image_sim import dark_current


## A dark frame measures dark current

Recall that *dark current* refers to counts (electrons) generated in a pixel because an electron in the pixel happens to have enough energy to "break free" and register as a count. The distribution of electron thermal energies in  pixel follows a [Maxwell-Boltzmann distribution](https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution) in which most electrons have energy around $kT$, where $T$ is the temperature of the sensor and $k$ is the Boltzmann constant. There is a distribution of energies, though, and occasionally an electron will be high enough energy to jump to the conducting band in the chip, registering the same as an electron excited by a photon. Since the Maxwell-Boltzmann distribution depends on temperature the rate at which dark current appears in a pixel is also expected to depend on temperature. 

A *dark frame* (also called a *dark image*) is an image taken with your camera with the shutter closed. It is the sum of the bias level of your camera, the readout noise, and the dark current.

You measure the dark current in your camera by taking dark frames.

## Dark current theory

The expected signal in a dark frame exposure of time $t$ is proportional to $t$. If we call the dark electrons in an exposure $d_e(t)$ and the dark current $d_c(T)$, where $T$ is the temperature, then 

$$
d_e(t) = d_c(T) t.
$$

For liquid-cooled cameras, particularly ones cooled bu liquid nitrogen, the operating temperature doesn't change. For thermo-electrically cooled cameras one is able to set the desired operating temperature. As a result, you should be able to ignore the temperature dependence of the dark current.

The thermo-electric coolers can usually cool by some fixed amount below the ambient temperature. Though in principle one could choose to always cool by the same fixed amount, like $50^\circ$C below the ambient temperature, there is an advantage to always running your camera at the same temperature: dark frames taken on one date are potentially useful on another date. If the operating temperature varies then you need to make sure to take dark frames every time you observe unless you carefully characterize the temperature dependence of your dark current. 

It will turn out that for practical reasons -- not all pixels in your camera have the same dark current -- it is usually best to take dark frames every time you observe anyway.

### Illustration

For the purposes of illustrating some of the properties of dark current and dark frames we'll generated some simulated images in which the counts are due to dark current alone. We'll use these values:

+ Dark current is $d_c(T) = 0.01 e^-$/pixel/sec
+ Gain is $g = 1.5 e^-$/ADU

In [ ]:
dark_rate = 0.01
gain = 1.5

#### Dark current is a random process

The dark counts in a dark frame are counts and so they follow a Poisson distribution. The plot below shows the dark current in the pixel `[100, 100]` in 20 different simulated images each with exposure time 100 sec. Note that the counts vary from image to image but that the average is very close to the expected value.

The expected value of the dark counts for this image are $d_e(t)/g = 0.667~$counts.

In [ ]:
n_images = 20
n_pixels = 20
pixels = np.random.randint(50, high=190, size=n_pixels)
pixel_values = np.zeros(n_images)
pixel_averages = np.zeros(n_images)
exposure = 100
base_image = np.zeros([200, 200])

plt.figure(figsize=(20, 10))
for pixel in pixels:
    for n in range(n_images):
        a_dark = dark_current(base_image, dark_rate, exposure, gain=gain, hot_pixels=False)
        pixel_values[n] = a_dark[pixel, pixel]

    plt.plot(pixel_values, label='pixel [{0}, {0}]'.format(pixel), alpha=0.5)
    pixel_averages += pixel_values

plt.plot(pixel_averages / n_pixels, 
         linewidth=3,
         label='Average over {} pixels'.format(n_pixels))
plt.xlabel('Image number')
plt.ylabel('Counts due to dark current')
plt.xlim(0, n_images - 1)
plt.legend()